In [1]:
%pwd

'/home/qula0496/quan/Nonstationary_Transformers/ns_models'

In [2]:
%cd ..

/home/qula0496/quan/Nonstationary_Transformers


/home/qula0496/quan/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
import os
gpu_id = '2'
n_users = 10
os.environ['CUDA_VISIBLE_DEVICES'] = gpu_id
dataset_name = 'milan'
sub_set_name = 'sms'
series_type = 'hourly'

In [4]:
from utils.data_utils import *
from utils.model_utils import *
# from utils.koopman_utils import *
from modules.serverbase import *
from modules.userbase import *
from modules.servernsTransformer import *
from modules.usernsTranformer import *
import numpy as np
import torch
torch.cuda.init()

In [5]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
def preprocess(data, std=True):
    '''
    Function to do preprocess for input data
    inputs:
        + data: pandas.DataFrame
    outputs:
        + preprocessed_data: pandas.DataFrame
    '''
    if std:
        scaler = StandardScaler()
        temp = scaler.fit_transform(data)
    else:
        scaler = MinMaxScaler()
        temp = scaler.fit_transform(data)
    processed_data = pd.DataFrame(temp, index=data.index, columns = data.columns)
    # processed_data = processed_data.to_numpy()
    return processed_data

In [6]:
file_path = f"./dataset/{dataset_name}/{dataset_name}_{sub_set_name}_{series_type}.csv"
raw_data = pd.read_csv(file_path)
raw_data = raw_data.pivot(index='startTime', columns='gridID', values=sub_set_name)
raw_data = raw_data.fillna(0)
raw_data = preprocess(raw_data, std=False)
raw_data.head()

gridID,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.058891,0.057828,0.056731,0.062167,0.060549,0.056731,0.056731,0.056731,0.056731,0.040944,...,0.080203,0.101781,0.150701,0.135242,0.155147,0.129418,0.099168,0.105147,0.138711,0.098340
2013-11-01 01:00:00,0.035490,0.035289,0.035082,0.036110,0.035334,0.035082,0.035082,0.035082,0.035082,0.029524,...,0.067121,0.067424,0.061588,0.050906,0.036478,0.026777,0.027318,0.027212,0.027128,0.029307
2013-11-01 02:00:00,0.012792,0.012636,0.012475,0.013273,0.013126,0.012475,0.012475,0.012475,0.012475,0.010654,...,0.064548,0.055739,0.040941,0.031525,0.031755,0.024837,0.025149,0.024995,0.030861,0.043975
2013-11-01 03:00:00,0.037618,0.037160,0.036688,0.039028,0.036827,0.036688,0.036688,0.036688,0.036688,0.021672,...,0.016029,0.023516,0.044500,0.049647,0.030611,0.019703,0.021844,0.021387,0.017950,0.020467
2013-11-01 04:00:00,0.015582,0.015581,0.015580,0.015585,0.015074,0.015580,0.015580,0.015580,0.015580,0.012785,...,0.017414,0.013548,0.009676,0.015734,0.019272,0.021963,0.024705,0.024309,0.021681,0.024750


In [7]:
# prompt: write pandas to csv

import pandas as pd

# Save the DataFrame to a CSV file
raw_data.rename(columns={'startTime': 'date'}, inplace=True)
raw_data.to_csv(f"./dataset/{dataset_name}/{sub_set_name}.csv", index_label='date')
raw_data.head()

gridID,1,2,3,4,5,6,7,8,9,10,...,9991,9992,9993,9994,9995,9996,9997,9998,9999,10000
startTime,,,,,,,,,,,,,,,,,,,,,
2013-11-01 00:00:00,0.058891,0.057828,0.056731,0.062167,0.060549,0.056731,0.056731,0.056731,0.056731,0.040944,...,0.080203,0.101781,0.150701,0.135242,0.155147,0.129418,0.099168,0.105147,0.138711,0.098340
2013-11-01 01:00:00,0.035490,0.035289,0.035082,0.036110,0.035334,0.035082,0.035082,0.035082,0.035082,0.029524,...,0.067121,0.067424,0.061588,0.050906,0.036478,0.026777,0.027318,0.027212,0.027128,0.029307
2013-11-01 02:00:00,0.012792,0.012636,0.012475,0.013273,0.013126,0.012475,0.012475,0.012475,0.012475,0.010654,...,0.064548,0.055739,0.040941,0.031525,0.031755,0.024837,0.025149,0.024995,0.030861,0.043975
2013-11-01 03:00:00,0.037618,0.037160,0.036688,0.039028,0.036827,0.036688,0.036688,0.036688,0.036688,0.021672,...,0.016029,0.023516,0.044500,0.049647,0.030611,0.019703,0.021844,0.021387,0.017950,0.020467
2013-11-01 04:00:00,0.015582,0.015581,0.015580,0.015585,0.015074,0.015580,0.015580,0.015580,0.015580,0.012785,...,0.017414,0.013548,0.009676,0.015734,0.019272,0.021963,0.024705,0.024309,0.021681,0.024750


In [8]:
# Download the dataset
# download_elec_dataset()

# # Clean dataset
# elec_data = clean_elec()

# from sklearn.preprocessing import StandardScaler, MinMaxScaler
# Std_normalization = 1
# if Std_normalization:
#     scaler = StandardScaler()
#     temp = scaler.fit_transform(elec_data)
#     norm_means = scaler.mean_
#     norm_std = scaler.scale_
# else:
#     scaler = MinMaxScaler()
#     temp = scaler.fit_transform(elec_data)

# elec_data = pd.DataFrame(elec_data, index=elec_data.index, columns = elec_data.columns)

In [9]:
from ns_models.ns_TransformerConfig import NS_TransformerConfig
user_num_ts = int(raw_data.shape[1] / n_users)
args = NS_TransformerConfig()
args.devices = gpu_id
args.enc_in = user_num_ts
args.dec_in = user_num_ts
args.c_out = user_num_ts
args.root_path='./dataset/milan/'
args.data_path='sms.csv'
args.data_path

'sms.csv'

In [10]:
from torch.utils.data import DataLoader
from data_provider.data_factory import *
from data_provider.data_loader import *

test_data, test_loader = data_provider(args, flag='test')

In [11]:
test_data[0][0].shape

(96, 10000)

In [12]:
user_data_list = []
server_data_list = []
for i in range(n_users):
    train_data, train_loader = data_provider(args, flag='train', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    test_data, test_loader = data_provider(args, flag='test', start=i*user_num_ts+1, end=min(user_num_ts*(i+1)+1, raw_data.shape[1]+1))
    # user_data = train_set.filter(lambda e, idx: idx>=(i*user_num_ts) and idx < user_num_ts*(i+1), with_indices=True)
    user_data_list.append(train_loader)
    server_data_list.append(test_loader)
    # print(train_data[0][0].shape)

In [13]:
from ns_models import ns_Transformer
server_model = ns_Transformer.Model(configs=args)
server_model

Model(
  (enc_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1000, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (dec_embedding): DataEmbedding(
    (value_embedding): TokenEmbedding(
      (tokenConv): Conv1d(1000, 64, kernel_size=(3,), stride=(1,), padding=(1,), bias=False, padding_mode=circular)
    )
    (position_embedding): PositionalEmbedding()
    (temporal_embedding): TimeFeatureEmbedding(
      (embed): Linear(in_features=4, out_features=64, bias=False)
    )
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): Encoder(
    (attn_layers): ModuleList(
      (0-5): 6 x EncoderLayer(
        (attention): AttentionLayer(
          (inner_attention): DSAttention(
      

# Setup WANDB

In [14]:
import wandb
import torch

NUM_GPUS = torch.cuda.device_count()
LR = 1e-3
GLOBAL_EPOCHS = 50
LOCAL_EPOCHS = 5
BATCH_SIZE = 128
L2_PENALTY = 0.0
USER_RATIO = 0.2
run = wandb.init(
    # project name
    project="Federated Non-Stationary Transformer on Milan dataset",
    # experiment name
    name=f"Federated Non-Stationary Transformer on Milan SMS dataset",
    # Hyperparams
    config={
        "dataset": "Milan Call",
        "preprocess_type": "std",
        "num_user": n_users,
        "learning_rate": LR,
        "global_epochs": GLOBAL_EPOCHS,
        "local_epochs": LOCAL_EPOCHS,
        "batch_size": BATCH_SIZE,
        "num_gpus": NUM_GPUS,
        "user_ratio": USER_RATIO,
        "l2_penalty": L2_PENALTY,
        "total_time_series": 10000,
        "detrending_data": "No"
    })

config_wanb = wandb.config

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kenlvq (quanla). Use `wandb login --relogin` to force relogin


In [15]:
import random
from tqdm import tqdm
from math import sqrt

server = ServerNsTransformer(model=server_model, test_loader=server_data_list)

user_list = []

# Create users
for i in range(config_wanb.num_user):
    user_i = UserNsTransformer(train_loader=user_data_list[i], model=server_model, user_id=i, local_epochs=config_wanb.local_epochs)
    user_list.append(user_i)

for _ in tqdm(range(config_wanb.global_epochs), desc=f"Progress"):
    # Distribute initial model to users
    server.distribute_model(user_list)
    
    # Sub-sample users
    sub_user_list = random.sample(user_list, int(config_wanb.user_ratio * config_wanb.num_user))

    # Check the sub-sampled user and train model
    users_loss = 0.0
    for user in sub_user_list:
        print(user)
        user_loss = user.user_train(args)
        users_loss += user_loss
    # Aggregate weights on server
    server.aggregate_weights(sub_user_list)

    # Calulate avg loss on selected users
    train_loss =  users_loss / len(sub_user_list)

    total_mae = []
    total_mse = []
    for test_loader in server.test_loader:    
        mae, mse, rmse, mape, mspe = server.model_eval(args=args, test_loader=test_loader)
        total_mae.append(mae)
        total_mse.append(mse)
    
    wandb.log({"train_loss": train_loss, "mae": sum(total_mae)/len(total_mae), 'rmse': sqrt(sum(total_mse)/len(total_mse))})

Progress:   0%|          | 0/50 [00:00<?, ?it/s]

Epoch: 1 cost time: 4.811601161956787
Epoch: 2 cost time: 3.7105960845947266
Epoch: 3 cost time: 3.439992904663086
Epoch: 4 cost time: 3.235430955886841
Epoch: 5 cost time: 3.2645459175109863
Epoch: 1 cost time: 3.4284677505493164
Epoch: 2 cost time: 3.305551290512085
Epoch: 3 cost time: 4.155806541442871
Epoch: 4 cost time: 4.127667665481567
Epoch: 5 cost time: 3.406243324279785


Progress:   2%|▏         | 1/50 [00:53<43:24, 53.15s/it]

Epoch: 1 cost time: 2.9220876693725586
Epoch: 2 cost time: 2.6818013191223145
Epoch: 3 cost time: 2.7936038970947266
Epoch: 4 cost time: 2.8085532188415527
Epoch: 5 cost time: 2.5688014030456543
Epoch: 1 cost time: 2.6302552223205566
Epoch: 2 cost time: 2.8772761821746826
Epoch: 3 cost time: 3.3885507583618164
Epoch: 4 cost time: 3.1816656589508057
Epoch: 5 cost time: 3.5140602588653564


Progress:   4%|▍         | 2/50 [01:38<38:50, 48.56s/it]

...

Progress:  96%|█████████▌| 48/50 [36:41<01:30, 45.07s/it]

Epoch: 1 cost time: 3.1045804023742676
Epoch: 2 cost time: 3.3459949493408203
Epoch: 3 cost time: 2.593489170074463
Epoch: 4 cost time: 2.77573561668396
Epoch: 5 cost time: 2.714428424835205
Epoch: 1 cost time: 2.8254282474517822
Epoch: 2 cost time: 2.6073262691497803
Epoch: 3 cost time: 2.8610143661499023
Epoch: 4 cost time: 2.6201136112213135
Epoch: 5 cost time: 2.6563363075256348


Progress:  98%|█████████▊| 49/50 [37:26<00:45, 45.05s/it]

Epoch: 1 cost time: 3.4713056087493896
Epoch: 2 cost time: 3.010624647140503
Epoch: 3 cost time: 2.35121750831604
Epoch: 4 cost time: 2.6364457607269287
Epoch: 5 cost time: 2.8902852535247803
Epoch: 1 cost time: 2.933939218521118
Epoch: 2 cost time: 3.056758165359497
Epoch: 3 cost time: 2.8902742862701416
Epoch: 4 cost time: 3.093055248260498
Epoch: 5 cost time: 2.953787088394165


Progress: 100%|██████████| 50/50 [38:04<00:00, 45.69s/it]
